In [1]:
from abc import ABC, ABCMeta, abstractmethod
from typing import Callable, TypeVar, Optional, Any,  Type
T = TypeVar("T")

class Adapter:
    """
    Adapts an object by replacing methods.
    
    Usage:
        Adapter(adaptee desired_method=adaptee.method)
    """
    def __init__(self, obj: T, **adapted_methods: Callable):
        # Setting the adapted methods in the object's dict.
        self.obj = obj
        self.__dict__.update(adapted_methods)

    def __getattr__(self, attr):
        """All non-adapted calls are passed to the object."""
        return getattr(self.obj, attr)

    def original_dict(self):
        return self.obj.__dict__

class Singleton(ABCMeta):
    """
    Class of that type can be initialized only once. 
    """
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls.__name__ not in cls._instances:
            cls._instances[cls.__name__] = super().__call__(*args, **kwargs)
        return cls._instances[cls.__name__]





In [2]:
from typing import Optional, TypeVar, Callable
T = TypeVar("T")

class Module:
    _successor = None
    def __init__(self, name: str) -> None:
        self.name = name
        
    
    def handle(self) -> None:
        res = self.run()
        if not self._successor:
            return res
        else:
            self._successor.handle(res)
            
    def run(self) -> None:
        raise NotImplementedError

class Device(Module):
    def __init__(self,  name: str, handler: Callable) -> None:
        self.handler=handler
    def run(self) -> None:
        return self.handler()

def beep(*args, **kwargs):
    pass


In [3]:
try:
    print("Loading visa package (it takes a while)")
    import visa

    print("Connecting to a device\n")
    rm = visa.ResourceManager('@py')
    smu2450 = rm.open_resource('USB0::1510::9296::04384536::0::INSTR')
    smu2450.timeout = None
    dmm6500 = rm.open_resource('USB0::1510::25856::04507180::0::INSTR')
    dmm6500.timeout = None
    beep('C')

    print(smu2450.query('*IDN?'))
    print(dmm6500.query('*IDN?'))
    print(dmm6500.query('SYST:CARD1:IDN?'))

    print("\nSetting it up (this is quite fast)")

    smu2450.write('*RST')
    smu2450.write('OUTP:SMOD HIMP')
    smu2450.write('SOUR:FUNC CURR')
    smu2450.write('SOUR:CURR:VLIM .5')
    smu2450.write('SOUR:DEL .1')

    beep('CEC')
except Exception as e:
    print(repr(e))

Loading visa package (it takes a while)
Connecting to a device

KEITHLEY INSTRUMENTS,MODEL 2450,04384536,1.6.4c

KEITHLEY INSTRUMENTS,MODEL DMM6500,04507180,1.7.5b

2000,10-Chan Mux,0.0.0a,00000000


Setting it up (this is quite fast)


-5.004972e-07

In [ ]:
def voltMeas():
    dmm6500.write('*RST')
    dmm6500.write('SENS:FUNC "VOLT:DC", (@1)')
    dmm6500.write('SENS:VOLT:RANG 0.1, (@1)')
    dmm6500.write('SENS:VOLT:INP AUTO, (@1)')
    dmm6500.write('SENS:VOLT:NPLC 10, (@1)')
    dmm6500.write('SENS:VOLT:AZER ON, (@1)')
    dmm6500.write('ROUT:CLOS (@1)')

    V = dmm6500.query('READ?')
    return float(V)*(-1)

def resMeas():
    smu2450.write('SOURCE:CURR 0.01')
    smu2450.write('OUTP ON')
    dmm6500.write('*RST')
    dmm6500.write('SENS:FUNC "VOLT:DC", (@1)')
    dmm6500.write('SENS:VOLT:RANG 0.1, (@1)')
    dmm6500.write('SENS:VOLT:INP AUTO, (@1)')
    dmm6500.write('SENS:VOLT:NPLC 10, (@1)')
    dmm6500.write('SENS:VOLT:AZER ON, (@1)')
    dmm6500.write('ROUT:CLOS (@1)')

    V1=float(dmm6500.query('READ?'))
    smu2450.write('OUTP OFF')

    smu2450.write('SOURCE:CURR -0.01')
    smu2450.write('OUTP ON')
    dmm6500.write('*RST')
    dmm6500.write('SENS:FUNC "VOLT:DC", (@1)')
    dmm6500.write('SENS:VOLT:RANG 0.1, (@1)')
    dmm6500.write('SENS:VOLT:INP AUTO, (@1)')
    dmm6500.write('SENS:VOLT:NPLC 10, (@1)')
    dmm6500.write('SENS:VOLT:AZER ON, (@1)')
    dmm6500.write('ROUT:CLOS (@1)')

    V2=float(dmm6500.query('READ?'))
    smu2450.write('OUTP OFF')

    R=(V1-V2)/.02
    return float(R)

In [11]:

d=Device("dmm6500", voltMeas)
e=Device("smu2450", resmeas)
print("{},{}".format(d.handle(), d.__class__.mro())
e.handle()



-3.217777e-07